#### Importing python packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflowjs as tfjs

tf.enable_eager_execution()

#### Loading the MNIST database

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Defining the model

In [ ]:
HIDDEN_LAYERS = 128
DROPOUT_RATE = 0.2

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28, 28)),
                                    tf.keras.layers.Dense(HIDDEN_LAYERS, activation='relu'),
                                    tf.keras.layers.Dropout(DROPOUT_RATE),
                                    tf.keras.layers.Dense(10, activation='softmax')])

In [ ]:
model.summary()

## Defining training

In [ ]:
LEARNING_RATE = 0.0001

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Training the model

In [ ]:
VALIDATION_SPLIT = 0.4
EPOCHS = 10
BATCH_SIZE = 32

In [ ]:
history = model.fit(x_train,
                    y_train,
                    validation_split=VALIDATION_SPLIT,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)

## Plotting training curves

In [ ]:
def plot_learning_curves(history: tf.keras.callbacks.History):
    history = history.history
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

    ax1.title.set_text('Loss curves')
    ax1.plot(range(1, 11),
             history['loss'],
             label='loss')
    ax1.plot(range(1, 11),
             history['val_loss'],
             label='val_loss')
    ax1.legend()

    ax2.title.set_text('Accurracy curves')
    ax2.plot(history['acc'],
             label='acc')
    ax2.plot(history['val_acc'],
             label='val_acc')
    ax2.legend()

In [ ]:
plot_learning_curves(history)

## Testing the model

In [ ]:
model.evaluate(x_train, y_train, verbose=2)

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

## Saving the model for JS

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, './model')